In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
train[train == -1] = np.nan

In [4]:
### just drop nas
train_no_nans = train.dropna()

In [47]:
cat_cols = [col for col in train_no_nans.columns if col.endswith('cat')]
cat_cols_maxs = train_no_nans[cat_cols].max(axis=0)
truly_cat_cols = cat_cols_maxs[cat_cols_maxs > 1].index.tolist()
for col in truly_cat_cols:
    train_no_nans[col] = train_no_nans[col].astype(int).astype(str)
ohe_train_no_nans = pd.get_dummies(train_no_nans)

ValueError: invalid literal for int() with base 10: '1.0'

In [6]:
from sklearn.model_selection import train_test_split
X, y = ohe_train_no_nans.iloc[:, 2:].values, ohe_train_no_nans.iloc[:, 1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [8]:
class_w = len(y_train) / (2 * np.bincount(y_train))
print(class_w)
mapper_fn = np.vectorize({0: class_w[0], 1: class_w[1]}.get)
weighted_y_train = mapper_fn(y_train)
weighted_y_test = mapper_fn(y_test)

[  0.52380296  11.00289381]


In [ ]:
# from sklearn.linear_model import LogisticRegression
# for c in np.arange(-4, 6, dtype=float):
#     lr = LogisticRegression(penalty='l1', C=10**c, random_state=0, class_weight='balanced')
#     lr.fit(X_train_std, y_train)
#     print('regularization C is ' + str(c))
#     print('Training accuracy:', lr.score(X_train_std, y_train, sample_weight = weighted_y_train))
#     print('Test accuracy:', lr.score(X_test_std, y_test, sample_weight = weighted_y_test))

regularization C is -4.0
Training accuracy: 0.5
Test accuracy: 0.501107580208
regularization C is -3.0
Training accuracy: 0.586783587913
Test accuracy: 0.57655576976
regularization C is -2.0
Training accuracy: 0.603211772517
Test accuracy: 0.579449262796
regularization C is -1.0
Training accuracy: 0.604487295709
Test accuracy: 0.576199031901
regularization C is 0.0
Training accuracy: 0.603989979454
Test accuracy: 0.575946884666


In [10]:
#picked C = 10**-2 from above
#regularization C is -2.0
#Training accuracy: 0.603211772517
#Test accuracy: 0.579449262796


In [11]:
stdsc = StandardScaler()
X_std = stdsc.fit_transform(X)
full_class_w = len(y) / (2 * np.bincount(y))

from sklearn.linear_model import LogisticRegression
lr_fin = LogisticRegression(penalty='l1', C=10**(-2), random_state=0, class_weight='balanced')
lr_fin.fit(X_std, y)

LogisticRegression(C=0.01, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=0,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [32]:
from sklearn.preprocessing import Imputer
cat_cols = [col for col in train.columns if (col.endswith('cat') or col.endswith('bin'))]
num_cols = [col for col in train.columns if not ((col.endswith('cat') or col.endswith('bin')) or (col in ['target', 'id']))]

cat_part = train[cat_cols]
num_part = train[num_cols]

cat_imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
cat_imp.fit(cat_part)
num_imp = Imputer(missing_values='NaN', strategy='median', axis=0)
num_imp.fit(num_part)

['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin', 'ps_calc_20_bin']
['ps_ind_01', 'ps_ind_03', 'ps_ind_14', 'ps_ind_15', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14']


Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)

In [46]:
pd.set_option('display.max_columns', 500)

cat_imputed = pd.DataFrame(cat_imp.transform(test[cat_cols]), columns=cat_cols)
for col in cat_imputed.columns:
    cat_imputed[col] = cat_imputed[col].astype(int)
num_imputed = pd.DataFrame(num_imp.transform(test[num_cols]), columns=num_cols)
combined = pd.concat([cat_imputed, num_imputed], axis=1)
for col in truly_cat_cols:
    combined[col] = combined[col].astype(str)
ohe_test_no_nans = pd.get_dummies(combined)
ohe_train_no_nans
# imputed_ohe_test = ohe_test_no_nans[ohe_train_no_nans.columns[2:]]

,id,target,ps_ind_01,ps_ind_03,ps_ind_04_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_02_cat,ps_car_03_cat,ps_car_05_cat,ps_car_07_cat,ps_car_08_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,ps_ind_02_cat_1.0,ps_ind_02_cat_2.0,ps_ind_02_cat_3.0,ps_ind_02_cat_4.0,ps_ind_05_cat_0.0,ps_ind_05_cat_1.0,ps_ind_05_cat_2.0,ps_ind_05_cat_3.0,ps_ind_05_cat_4.0,ps_ind_05_cat_5.0,ps_ind_05_cat_6.0,ps_car_01_cat_0.0,ps_car_01_cat_1.0,ps_car_01_cat_10.0,ps_car_01_cat_11.0,ps_car_01_cat_2.0,ps_car_01_cat_3.0,ps_car_01_cat_4.0,ps_car_01_cat_5.0,ps_car_01_cat_6.0,ps_car_01_cat_7.0,ps_car_01_cat_8.0,ps_car_01_cat_9.0,ps_car_04_cat_0,ps_car_04_cat_1,ps_car_04_cat_2,ps_car_04_cat_3,ps_car_04_cat_4,ps_car_04_cat_5,ps_car_04_cat_6,ps_car_04_cat_7,ps_car_04_cat_8,ps_car_04_cat_9,ps_car_06_cat_0,ps_car_06_cat_1,ps_car_06_cat_10,ps_car_06_cat_11,ps_car_06_cat_12,ps_car_06_cat_13,ps_car_06_cat_14,ps_car_06_cat_15,ps_car_06_cat_16,ps_car_06_cat_17,ps_car_06_cat_2,ps_car_06_cat_3,ps_car_06_cat_4,ps_car_06_cat_5,ps_car_06_cat_6,ps_car_06_cat_7,ps_car_06_cat_8,ps_car_06_cat_9,ps_car_09_cat_0.0,ps_car_09_cat_1.0,ps_car_09_cat_2.0,ps_car_09_cat_3.0,ps_car_09_cat_4.0,ps_car_10_cat_0,ps_car_10_cat_1,ps_car_10_cat_2,ps_car_11_cat_1,ps_car_11_cat_10,ps_car_11_cat_100,ps_car_11_cat_101,ps_car_11_cat_102,ps_car_11_cat_103,ps_car_11_cat_104,ps_car_11_cat_11,ps_car_11_cat_12,ps_car_11_cat_13,ps_car_11_cat_14,ps_car_11_cat_15,ps_car_11_cat_16,ps_car_11_cat_17,ps_car_11_cat_18,ps_car_11_cat_19,ps_car_11_cat_20,ps_car_11_cat_21,ps_car_11_cat_22,ps_car_11_cat_23,ps_car_11_cat_24,ps_car_11_cat_26,ps_car_11_cat_27,ps_car_11_cat_28,ps_car_11_cat_29,ps_car_11_cat_3,ps_car_11_cat_30,ps_car_11_cat_31,ps_car_11_cat_32,ps_car_11_cat_33,ps_car_11_cat_34,ps_car_11_cat_35,ps_car_11_cat_36,ps_car_11_cat_37,ps_car_11_cat_38,ps_car_11_cat_39,ps_car_11_cat_4,ps_car_11_cat_40,ps_car_11_cat_41,ps_car_11_cat_42,ps_car_11_cat_43,ps_car_11_cat_44,ps_car_11_cat_45,ps_car_11_cat_46,ps_car_11_cat_47,ps_car_11_cat_48,ps_car_11_cat_49,ps_car_11_cat_5,ps_car_11_cat_50,ps_car_11_cat_51,ps_car_11_cat_52,ps_car_11_cat_53,ps_car_11_cat_54,ps_car_11_cat_55,ps_car_11_cat_56,ps_car_11_cat_57,ps_car_11_cat_58,ps_car_11_cat_59,ps_car_11_cat_6,ps_car_11_cat_60,ps_car_11_cat_61,ps_car_11_cat_62,ps_car_11_cat_63,ps_car_11_cat_64,ps_car_11_cat_65,ps_car_11_cat_66,ps_car_11_cat_67,ps_car_11_cat_68,ps_car_11_cat_69,ps_car_11_cat_7,ps_car_11_cat_70,ps_car_11_cat_71,ps_car_11_cat_72,ps_car_11_cat_73,ps_car_11_cat_74,ps_car_11_cat_75,ps_car_11_cat_76,ps_car_11_cat_77,ps_car_11_cat_78,ps_car_11_cat_79,ps_car_11_cat_8,ps_car_11_cat_81,ps_car_11_cat_82,ps_car_11_cat_83,ps_car_11_cat_84,ps_car_11_cat_85,ps_car_11_cat_86,ps_car_11_cat_87,ps_car_11_cat_88,ps_car_11_cat_89,ps_car_11_cat_9,ps_car_11_cat_90,ps_car_11_cat_91,ps_car_11_cat_92,ps_car_11_cat_93,ps_car_11_cat_94,ps_car_11_cat_95,ps_car_11_cat_96,ps_car_11_cat_97,ps_car_11_cat_98,ps_car_11_cat_99
3,16,0,0,2,0.0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,1.0,0.0,1.0,1.0,1,1.0,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,22,0,5,4,0.0,1,0,0,0,0,0,0,0,0,13,1,0,0,0.7,0.4,0.607248,1.0,0.0,0.0,1.0,1,3.0,0.446990,0.900574,0.374833,3.316625,0.1,0.5,0.1,1,2,7,1,6,1,13,7,1,3,6,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0